In [2]:
# Import the necessary libraries 
import pandas as pd
import numpy as np

In [1]:
# Read the data
data = pd.read_csv('dataset/combine/combined-csv-files.csv')

# Randomised the data to 8000 rows (from the original 150000 rows)
data.sample(8000).to_csv('dataset/combine/combined-csv-files-sample.csv', index=False)

C:\Users\putty\AppData\Local\Temp/ipykernel_25680/2622503571.py:4: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset/combine/combined-csv-files.csv')


In [6]:
# Loading the 8000 rows dataset 

data = pd.read_csv('dataset/combine/combined-csv-files-sample.csv')

# Checking the data
data.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,535b0db761369fc82e6e20ffbf0c2961,2022-04-19,2022-06-06,LGA,LAX,QAA0KKEN,PT8H46M,0.0,False,False,...,1654550880||1654576560,2022-06-06T17:28:00.000-04:00||2022-06-06T21:3...,IAD||LAX,LGA||IAD,United||United,UA||UA,Embraer 175 (Enhanced Winglets)||Boeing 737-900,5880||19680,221||2295,coach||coach
1,ae2ee04df190b4a3b3d86de847860ecc,2022-04-22,2022-05-23,DEN,DTW,K0AIZNN1,PT6H33M,0.0,False,False,...,1653300420||1653312180,2022-05-23T06:07:00.000-04:00||2022-05-23T09:2...,CLT||DTW,DEN||CLT,American Airlines||American Airlines,AA||AA,Airbus A321||Boeing 737-800,11820||6480,1346||505,coach||coach
2,2efe95e89f76ee18169f68553e3037cf,2022-04-22,2022-05-22,DFW,BOS,H0AJZNN1,PT3H45M,0.0,False,False,...,1653253740,2022-05-22T17:09:00.000-04:00,BOS,DFW,American Airlines,AA,Airbus A321,13500,1556,coach
3,86c9e9c2064348f556b7f0abd7352c4f,2022-04-16,2022-04-21,IAD,DEN,LA3OA0BG,PT6H31M,0.0,True,False,...,1650567840||1650585480,2022-04-21T15:04:00.000-04:00||2022-04-21T17:5...,DTW||DEN,IAD||DTW,Delta||Delta,DL||DL,Canadair Regional Jet 900||Airbus A321,5820||11580,391||1142,coach||coach
4,b53afb250f4fa14bdf01fb8031b154df,2022-04-19,2022-06-10,JFK,CLT,SVAHZNN1,PT4H37M,0.0,False,False,...,1654900080||1654911420,2022-06-10T18:28:00.000-04:00||2022-06-10T21:3...,ORF||CLT,JFK||ORF,American Airlines||American Airlines,AA||AA,Embraer 175||Canadair Regional Jet 900,5280||4980,301||290,coach||coach
